# Object Detection using yolov5 model pipeline with ZenML

ZenML Installation:

In [15]:
%pip install zenml
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
⠧ Installing integrations...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


{'status': 'ok', 'restart': True}

ZenML Setup:

In [ ]:
!rm -rf .zen

In [2]:
!zenml init

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
ZenML repository initialized at /content.
⠧ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.

The local active profile was initialized to 'default' and the local active stack
to 'default'. This local configuration will only take effect when you're running
ZenML from the initialized repository root, or from a subdirectory. For more 
information on profile and stack configuration, please visit 
https://docs.zenml.io/developer-guide/stacks-profiles-repositories.


In [3]:
!zenml profile create zenbytes

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'default' (local)
Error: Profile 'zenbytes' already exists.


In [4]:
!zenml profile set zenbytes

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'default' (local)
Active profile changed to: 'zenbytes'
⠋ Setting the active profile to 'zenbytes'...


In [5]:
!zenml stack set default

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'zenbytes' (local)
Active stack set to: 'default'
⠹ Setting the active stack to 'default'...


In [6]:
!zenml stack get

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'zenbytes' (local)
The active stack is: 'default'
⠋ Getting the active stack...


Defining object detection pipeline with ZenML:

In [7]:
from zenml.steps import step, Output
import os

@step
def environment_setup() -> None:
    """Setup the yolov5 environment"""
    print("Cloning the yolov5 repo")
    os.system('git clone https://github.com/ultralytics/yolov5')
    print("Change the current directory to yolov5")
    os.chdir(os.path.join(os.getcwd(), "yolov5"))
    print("Install all necessary requirements")
    os.system('pip install -r requirements.txt')

In [8]:
@step
def model_training()-> None:
    """Train the yolov5 model"""
    os.system("python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache")

In [9]:
@step
def model_inference() -> None:
    """Detect the model"""
    os.system("python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images")

In [10]:
@step
def model_validation() -> None:
    """Validate the model"""
    # Download COCO val
    torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
    os.system("unzip -q tmp.zip -d ../datasets && rm tmp.zip")  # unzip
    os.system("python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --hal")

In [12]:
from zenml.pipelines import pipeline
import torch

@pipeline
def object_detection_pipeline(setup, train, inference,validate):
    """Links all the steps together in a pipeline"""
    setup()
    train()
    inference()
    validate()

Running zenML Pipeline:

In [13]:
object_detection_instance = object_detection_pipeline(
    setup=environment_setup(), train=model_training(), inference=model_inference(), validate=model_validation()
)

In [14]:
object_detection_instance.run()

Creating run for pipeline: object_detection_pipeline
Cache enabled for pipeline object_detection_pipeline
Using stack default to run pipeline object_detection_pipeline...
Step environment_setup has started.
Using cached version of environment_setup.
Step environment_setup has finished in 0.044s.
Step model_inference has started.
Using cached version of model_inference.
Step model_inference has finished in 0.028s.
Step model_training has started.
Using cached version of model_training.
Step model_training has finished in 0.028s.
Step model_validation has started.


  0%|          | 0.00/780M [00:00<?, ?B/s]

Step model_validation has finished in 15.776s.
Pipeline run object_detection_pipeline-25_Aug_22-10_30_44_264855 has finished in 16.087s.
